<div style="display: table; width: 100%;">
  <div style="display: table-cell; text-align: center; vertical-align: middle; width: 70%;">
    <h1>Herramientas para Data Science</h1>
  </div>
  <div style="display: table-cell; text-align: center; vertical-align: middle; width: 30%;">
    <img src="https://github.com/UIDE-Tareas/6-Herramientas-Data-Science-Tarea1/blob/main/Assets/UideLogo.png?raw=true" alt="logo UIDE" style="width:50%;">
  </div>
</div>
<hr />

### 🟦 Componente Práctico 1  
🟡 Grupo: 3      
🟡 Semana: 1      
🟡 Docente:  Ing. Iván García S., PhD. (idgs78@hotmail.com)     

### 🟦 Realizado por:   
Estudiantes

💻 Evelin Rosero Ordoñez   

💻 Marjorie Muso Tandalla

💻 José Espinoza Bone

### 🟦 Objetivo y alcance del trabajo 
- Esta práctica tiene el objetivo de realizar un preprocesamiento de datos y un Análisis 
exploratorio de datos (estadísticas y visualización) de las variables más relevantes 
que considere para el propósito de predicción de supervivencia en el desastre del Titanic. Al finalizar la 
práctica, los maestrantes podrán manipular adecuadamente un dataset de datos 
estructurados (alfanuméricos). [Titanic - Machine Learning from Disaster](https://www.kaggle.com/competitions/titanic)

- Esta práctica tiene el objetivo de realizar la predicción de la supervivencia de los pasajeros del Titanic
(sobrevivió = 1, no sobrevivió = 0) utilizando los algoritmos de clasificación revisados de
regresión logística y K-NN, así como realizar la evaluación del rendimiento de los algoritmos usando las métricas y gráficas respectivas.
Al finalizar la práctica, los maestrantes podrán entender y manipular adecuadamente los diferentes parámetros e hiperparámetros de los algoritmos estudiados.

- Esta práctica tiene el objetivo de realizar la predicción de la supervivencia de los pasajeros del Titanic
(sobrevivió = 1, no sobrevivió = 0) utilizando los algoritmos de clasificación revisados de
árbol de decisión, bosque aleatorio y redes neuronales artificiales, así como realizar la
evaluación del rendimiento de los algoritmos usando las métricas y gráficas respectivas.
Al finalizar la práctica, los maestrantes podrán entender y manipular adecuadamente los diferentes parámetros e hiperparámetros de los algoritmos estudiados.


### 🟦 [Código fuente original](https://github.com/UIDE-Tareas/6-Herramientas-Data-Science-Tarea1.git)
Con [git](https://git-scm.com/) instalado. En Windows, Linux o MacOS ejecutar el comando.

```
git clone "https://github.com/UIDE-Tareas/6-Herramientas-Data-Science-Tarea1.git"
```






# 0️⃣ Preparar entorno

Funciones base para utilizar si son requeridas en el presente notebook. Adicional hay funciones utilitarias para utilizar con pandas.DataFrame y finalmente las funciones para cumplir con los objetivos del presente trabajo práctico.

In [ ]:
# UTILIDADES PARA GESTIÓN DE DEPENDENCIAS E INFORMACIÓN DEL ENTORNO

import sys
import subprocess
import os
from pathlib import Path
from enum import Enum
import zipfile
from typing import Optional
from typing import Iterable
from dataclasses import dataclass
from typing import cast
from typing import Tuple
from types import SimpleNamespace
from typing import Any
from typing import Protocol
from typing import Literal
from typing import Sequence

# Libs a instalar
LIBS = [
    "numpy",
    "pandas",
    "matplotlib",
    "seaborn",
    "scikit-learn",
    "requests",
    "wcwidth",
    "tensorflow"
]

class ConsoleColor(Enum):
    RED = "\033[91m"
    GREEN = "\033[92m"
    YELLOW = "\033[93m"
    BLUE = "\033[94m"
    MAGENTA = "\033[95m"
    CYAN = "\033[96m"
    WHITE = "\033[97m"
    RESET = "\033[0m"


def PrintColor(message: str, color: ConsoleColor) -> str:
    RESET = ConsoleColor.RESET.value
    return f"{color.value}{message}{RESET}"


def ShowMessage(
    message: str, title: str, icon: str, color: ConsoleColor, end: str = "\n"
):
    colored_title = PrintColor(icon + f"  " + title.upper() + ":", color)
    print(f"{colored_title} {message}", end=end)


def ShowInfoMessage(
    message: str, title: str = "Info", icon: str = "ℹ️", end: str = "\n"
):
    ShowMessage(message, title, icon, ConsoleColor.CYAN, end)


def ShowSuccessMessage(
    message: str, title: str = "Success", icon: str = "✅", end: str = "\n"
):
    ShowMessage(message, title, icon, ConsoleColor.GREEN, end)


def ShowErrorMessage(
    message: str, title: str = "Error", icon: str = "❌", end: str = "\n"
):
    ShowMessage(message, title, icon, ConsoleColor.RED, end)


def ShowWarningMessage(
    message: str, title: str = "Warning", icon: str = "⚠️", end: str = "\n"
):
    ShowMessage(message, title, icon, ConsoleColor.YELLOW, end)


# Funcion para ejecutar comandos
def RunCommand(
    commandList: list[str], printCommand: bool = True, printError: bool = True
) -> subprocess.CompletedProcess[str]:
    print("⏳", " ".join(commandList))

    if printCommand:
        proc = subprocess.Popen(
            commandList,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            bufsize=1,
            universal_newlines=True,
        )

        out_lines: list[str] = []
        assert proc.stdout is not None
        for line in proc.stdout:
            print(line, end="")
            out_lines.append(line)

        proc.wait()
        err_text = ""
        if proc.stderr is not None:
            err_text = proc.stderr.read() or ""

        if proc.returncode != 0 and printError and err_text:
            ShowErrorMessage(err_text, "", end="")
            # print(err_text, end="")

        return subprocess.CompletedProcess(
            args=commandList,
            returncode=proc.returncode,
            stdout="".join(out_lines),
            stderr=err_text,
        )

    else:
        result = subprocess.run(
            commandList, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
        )
        if result.returncode != 0 and printError and result.stderr:
            ShowErrorMessage(result.stderr, "", end="")
            # print(result.stderr, end="")
        return result


# Función para instalar las dependencias
def InstallDeps(libs: Optional[list[str]] = None):
    print("ℹ️ Installing deps.")
    printCommand = False
    printError = True
    RunCommand(
        [sys.executable, "-m", "pip", "install", "--upgrade", "pip"],
        printCommand=printCommand,
        printError=printError,
    )
    if libs is None or libs.count == 0:
        print("No hay elementos a instalar.")
    else:
        RunCommand(
            [sys.executable, "-m", "pip", "install", *libs],
            printCommand=printCommand,
            printError=printError,
        )
        print("Deps installed.")
    print()


# Función para mostrar info el ambiente de ejecución
def ShowEnvironmentInfo():
    print("ℹ️  Environment Info:")
    print("Python Version:", sys.version)
    print("Platform:", sys.platform)
    print("Executable Path:", sys.executable)
    print("Current Working Directory:", os.getcwd())
    print("VIRTUAL_ENV:", os.environ.get("VIRTUAL_ENV"))
    print("sys.prefix:", sys.prefix)
    print("sys.base_prefix:", sys.base_prefix)
    print()


InstallDeps(LIBS)
ShowEnvironmentInfo()
import requests


@dataclass(frozen=True)
class BoxStyle:
    TL: str
    TR: str
    BL: str
    BR: str
    H: str
    V: str

class TitleBoxLineStyle(Enum):
    SIMPLE = BoxStyle("┌", "┐", "└", "┘", "─", "│")
    DOUBLE = BoxStyle("╔", "╗", "╚", "╝", "═", "║")
    ROUNDED = BoxStyle("╭", "╮", "╰", "╯", "─", "│")
    HEAVY = BoxStyle("┏", "┓", "┗", "┛", "━", "┃")
    ASCII = BoxStyle("+", "+", "+", "+", "-", "|")
    DOUBLE_BOLD = BoxStyle("╔", "╗", "╚", "╝", "╬", "║")
    BLOCK = BoxStyle("█", "█", "█", "█", "█", "█")
    HEAVY_CROSS = BoxStyle("╒", "╕", "╘", "╛", "╪", "┃")
    METAL = BoxStyle("╞", "╡", "╘", "╛", "═", "║")


# Función para mostrar un título con recuadro
def ShowTitleBox(
    text: str,
    max_len: int = 100,
    boxLineStyle: TitleBoxLineStyle = TitleBoxLineStyle.SIMPLE,
    color: ConsoleColor = ConsoleColor.CYAN,
):
    try:

        def vislen(s: str) -> int:
            from wcwidth import wcswidth as _w

            n = _w(s)
            return n if n >= 0 else len(s)

    except Exception:

        def vislen(s: str) -> int:
            return len(s)

    pad = 1
    tlen = vislen(text)
    inner = max(max_len, tlen)
    left = (inner - tlen) // 2
    right = inner - tlen - left

    top = f"{boxLineStyle.value.TL}{boxLineStyle.value.H * (inner + 2 * pad)}{boxLineStyle.value.TR}"
    mid = f"{boxLineStyle.value.V}{' ' * pad}{' ' * left}{text}{' ' * right}{' ' * pad}{boxLineStyle.value.V}"
    bot = f"{boxLineStyle.value.BL}{boxLineStyle.value.H * (inner + 2 * pad)}{boxLineStyle.value.BR}"
    print(PrintColor("\n".join([top, mid, bot]), color))


# Función para descargar un archivo
def DownloadFile(uri: str, filename: str, overwrite: bool = False, timeout: int = 20, printInfo: bool = True):
    dest = Path(filename).resolve()
    if dest.exists() and dest.is_file() and dest.stat().st_size > 0 and not overwrite:
        if printInfo:
            print(
                f'✅ Ya existe: "{dest}". No se descarga (use overwrite=True para forzar).'
            )
        return
    if dest.parent and not dest.parent.exists():
        dest.parent.mkdir(parents=True, exist_ok=True)
    if printInfo:
        print(f'ℹ️ Descargando "{uri}" → "{dest}"')
    try:
        with requests.get(uri, stream=True, timeout=timeout) as resp:
            resp.raise_for_status()
            tmp = dest.with_suffix(dest.suffix + ".part")
            with open(tmp, "wb") as f:
                for chunk in resp.iter_content(chunk_size=1024 * 64):
                    if chunk:  # filtra keep-alive chunks
                        f.write(chunk)
            tmp.replace(dest)
        if printInfo: 
            print(f'✅ Archivo "{dest}" descargado exitosamente.')
    except requests.exceptions.RequestException as e:
        print(f"❌ Error al descargar: {e}")


# Función para descomprimir un archivo zip
def UnzipFile(filename: str, outputDir: str):
    print(f'ℹ️ Descomprimiendo "{filename}" en "{outputDir}"')
    try:
        with zipfile.ZipFile(filename, "r") as zip_ref:
            zip_ref.extractall(outputDir)
        print(f"Descomprimido en: {os.path.abspath(outputDir)}")
    except Exception as e:
        print(f"Error: {e}")


In [ ]:
# UTILIDADES PARA ANÁLISIS Y MANIPULACIÓN DE DATAFRAMES

# Importar libraries
import pandas as pd
import pandas
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from matplotlib.figure import Figure
from matplotlib.axes import Axes

from pandas import DataFrame
from pandas import Series
from sklearn.utils import Bunch
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    precision_score, recall_score, f1_score
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

warnings.filterwarnings("ignore")

# Configurar opciones de Pandas
pd.set_option("display.float_format", "{:.2f}".format)
pandas.set_option("display.max_rows", None)
pandas.set_option("display.max_columns", None)


# Función para mostrar la información del DataFrame.
def ShowDfInfo(df: pandas.DataFrame, title):
    display(f"ℹ️ INFO {title} ℹ️")
    df.info()
    display()


# Función para mostrar las n primeras filas del DataFrame.
def ShowDfHead(df: pandas.DataFrame, title: str, headQty=10):
    display(f"ℹ️ {title}: Primeros {headQty} elementos.")
    display(df.head(headQty))
    display()


# Función para mostrar las n últimas filas del DataFrame.
def ShowDfTail(df: pandas.DataFrame, title: str, tailQty=10):
    display(f"ℹ️ {title}: Últimos {tailQty} elementos.")
    display(df.tail(tailQty))
    display()


# Mostrar el tamaño del DataFrame
def ShowDfShape(df: pandas.DataFrame, title: str):
    display(f"ℹ️ {title} - Tamaño de los datos")
    display(f"{df.shape[0]} filas x {df.shape[1]} columnas")
    display()


# Función para mostrar la estadística descriptiva de todas las columnas del DataFrame, por tipo de dato.
def ShowDfStats(df: pandas.DataFrame, title: str = ""):
    display(f"ℹ️ Estadística descriptiva - {title}")
    numeric_cols = df.select_dtypes(include="number")
    if not numeric_cols.empty:
        display("    🔢 Columnas numéricas".upper())
        numeric_desc = (
            numeric_cols.describe().round(2).T
        )  # Transpuesta para añadir columna
        numeric_desc["var"] = numeric_cols.var(numeric_only=True).round(2)
        display(numeric_desc.T)
    non_numeric_cols = df.select_dtypes(
        include=["boolean", "string", "category", "object"]
    )
    if not non_numeric_cols.empty:
        display("    🔡 Columnas no numéricas".upper())
        non_numeric_desc = non_numeric_cols.describe()
        display(non_numeric_desc)
    datetime_cols = df.select_dtypes(include=["datetime", "datetimetz"])
    if not datetime_cols.empty:
        display("    📅 Columnas fechas".upper())
        datetime_desc = datetime_cols.describe()
        display(datetime_desc)


# Función para mostrar una visión general completa del DataFrame
def ShowFullDfOverview(df, title, headQty=5, tailQty=5):
    ShowDfInfo(df, title)
    ShowDfStats(df, title)
    ShowDfShape(df, title)
    ShowDfHead(df, title, headQty=headQty)
    ShowDfTail(df, title, tailQty=tailQty)


# Función para mostrar los valores nulos o NaN de cada columna en un DataFrame
def ShowDfNanValues(df: pandas.DataFrame, title: str):
    display(f"ℹ️ Contador de valores Nulos - {title}")
    nulls_count = df.isnull().sum()
    nulls_df = nulls_count.reset_index()
    nulls_df.columns = ["Columna", "Cantidad_Nulos"]
    display(nulls_df)
    display()


# Tipos de correlación
class CorrelationType(Enum):
    ALL = "all"
    STRONG = "strong"
    WEAK = "weak"


# Muestra las correlaciones completas, débiles y fuertes.
def ShowDfCorrelation(
    df: pandas.DataFrame,
    title: str,
    fig: Optional[Figure] = None,
    ax: Optional[Axes] = None,
    level: CorrelationType = CorrelationType.ALL,
    umbral: float = 0.6,
    showTable: bool = False,
    figsize: tuple = (8, 6),
    annotate: bool = True,
):
    if fig is None or ax is None:
        fig, ax = plt.subplots(figsize=figsize)

    display(f"ℹ️ {(title).upper()} - Matriz de Correlación, Type: {level.name}")

    corr = df.select_dtypes(include=["number"]).corr().copy()

    if level == CorrelationType.STRONG:
        corr = corr.where(np.abs(corr) >= umbral)
    elif level == CorrelationType.WEAK:
        corr = corr.where(np.abs(corr) < umbral)
        np.fill_diagonal(corr.values, 1)
    elif level != CorrelationType.ALL:
        raise ValueError(f"Invalid level: {level}")

    cax = ax.matshow(corr, vmin=-1, vmax=1)

    cols = corr.columns
    ax.set_xticks(range(len(cols)))
    ax.set_yticks(range(len(cols)))
    ax.set_xticklabels(cols, rotation=90, ha="left")
    ax.set_yticklabels(cols)

    fig.colorbar(cax)

    if annotate:
        for (i, j), value in np.ndenumerate(corr.values):
            if not np.isnan(value):
                ax.text(j, i, f"{value:+.2f}", ha="center", va="center")

    if level == CorrelationType.ALL:
        titulo = "Matriz de correlación completa"
    else:
        titulo = f"Matriz de correlación ({level.name}, umbral={umbral})"

    total_elementos = corr.size
    total_nodiagonal = corr.size - corr.shape[0]
    total_nan = corr.isna().sum().sum()
    total_validos = total_elementos - total_nan - corr.shape[0]

    titulo = (
        f"{titulo}, Total Matriz: {total_nodiagonal}, "
        f"Total válidos: {total_validos} ({((total_validos * 100) / total_nodiagonal):.2f}%)"
    )

    ax.set_title(titulo, pad=20)
    ax.grid(False)
    plt.tight_layout()
    plt.show()

    if showTable:
        display(corr)
    fig.show()
    return fig, corr


def NormalizeColumnNames(df: pandas.DataFrame) -> pandas.DataFrame:
    df.columns = [
        col.strip().title().replace(" ", "").replace("_", "") for col in df.columns
    ]
    return df


def DropColumns(
    df: pandas.DataFrame, toDrop: list[str], inplace: bool = False
) -> pandas.DataFrame:
    if not toDrop:
        return df
    if inplace:
        df.drop(columns=df.columns.intersection(toDrop), inplace=True)
        return df
    else:
        return df.drop(columns=df.columns.intersection(toDrop))


# Para almacenar los datos del dataset
@dataclass
class Dataset:
    X: pandas.DataFrame
    y: pandas.DataFrame


# Para almacenar los datos de split del dataset.
@dataclass
class DatasetSplit:
    Train: Dataset
    Test: Dataset


# Muestra el head de cada componente del split.
def ShowDatasetSplitHead(split: DatasetSplit, title: str, headQty: int = 5):
    ShowDfHead(split.Train.X, f"{title} - X Train", headQty)
    ShowDfHead(split.Train.y, f"{title} - y Train", headQty)
    ShowDfHead(split.Test.X, f"{title} - X Test", headQty)
    ShowDfHead(split.Test.y, f"{title} - y Test", headQty)


# Muestra la información del Dataset
def ShowDatasetInfo(data: Dataset, title):
    tAux = title
    title = f"{tAux} - Caracteristicas - X"
    ShowDfInfo(data.X, title)
    ShowDfShape(data.X, title)
    ShowDfStats(data.X, title)
    ShowDfNanValues(data.X, title)
    ShowDfHead(data.X, title)
    ShowDfTail(data.X, title)
    title = f"{tAux} - Características - y"
    ShowDfInfo(data.y, title)
    ShowDfShape(data.y, title)
    ShowDfStats(data.y, title)
    ShowDfNanValues(data.y, title)
    ShowDfHead(data.y, title)
    ShowDfTail(data.y, title)


# Muestra la información del Dataset Split
def ShowDatasetSplitInfo(split: DatasetSplit, title: str, headQty: int = 5):
    tAux = title
    title = f"{tAux} - TRAIN"
    ShowDatasetInfo(split.Train, title)
    title = f"{tAux} - TEST"
    ShowDatasetInfo(split.Test, title)


# Realiza el split del Dataset, en Train y test utilizando el ratio.
def SplitDataset(
    data: Dataset, trainRatio: float = 0.8, randomState: int = 42
) -> DatasetSplit:
    y_strat = data.y.iloc[:, 0]
    XTrain, XTest, yTrain, yTest = train_test_split(
        data.X,
        data.y,
        train_size=trainRatio,
        random_state=randomState,
        stratify=y_strat,
    )
    return DatasetSplit(
        Train=Dataset(X=XTrain.reset_index(drop=True), y=yTrain.reset_index(drop=True)),
        Test=Dataset(X=XTest.reset_index(drop=True), y=yTest.reset_index(drop=True)),
    )


# Contrato para los escaladores
class ScalerProtocol(Protocol):
    def fit(self, X, y: Any = None) -> Any: ...
    def transform(self, X) -> Any: ...
    def fit_transform(self, X, y: Any = None) -> Any: ...


# Para almacenar los datos del dataset aplicado el escalador.
@dataclass
class ScaledDatasetSplit(DatasetSplit):
    Scaler: ScalerProtocol

# Enum para los tipos de escaladores soportados
class ScalerType(Enum):
    STANDARD = "Standard"
    MIN_MAX = "minmax"
    ROBUST = "robust"
    MAX_ABS = "maxabs"
    NORMALIZER = "normalizer"
    QUANTILE = "quantile"
    POWER = "power"
    FUNCTION = "function"



# Crea una instancia de scaler según el Enum ScalerType.
def CreateScaler(scalerType: ScalerType, **kwargs) -> ScalerProtocol:
    if scalerType == ScalerType.STANDARD:
        return StandardScaler(**kwargs)
    if scalerType == ScalerType.MIN_MAX:
        return MinMaxScaler(**kwargs)
    if scalerType == ScalerType.ROBUST:
        return RobustScaler(**kwargs)
    if scalerType == ScalerType.MAX_ABS:
        return MaxAbsScaler(**kwargs)
    if scalerType == ScalerType.NORMALIZER:
        return Normalizer(**kwargs)
    if scalerType == ScalerType.QUANTILE:
        return QuantileTransformer(**kwargs)
    if scalerType == ScalerType.POWER:
        return PowerTransformer(**kwargs)
    if scalerType == ScalerType.FUNCTION:
        return FunctionTransformer(**kwargs)
    raise ValueError(f"ScalerType no soportado: {scalerType}")

def DetectScaler(scaler: ScalerProtocol) -> ScalerType:
    if isinstance(scaler, StandardScaler):
        return ScalerType.STANDARD
    if isinstance(scaler, MinMaxScaler):
        return ScalerType.MIN_MAX
    if isinstance(scaler, RobustScaler):
        return ScalerType.ROBUST
    if isinstance(scaler, MaxAbsScaler):
        return ScalerType.MAX_ABS
    if isinstance(scaler, Normalizer):
        return ScalerType.NORMALIZER
    if isinstance(scaler, QuantileTransformer):
        return ScalerType.QUANTILE
    if isinstance(scaler, PowerTransformer):
        return ScalerType.POWER
    if isinstance(scaler, FunctionTransformer):
        return ScalerType.FUNCTION
    raise ValueError(f"No se reconoce el tipo de scaler: {type(scaler)}")

# Escala el split usando el escalador proporcionado y retorna el split escalado.
def ScaleDatasetSplit(
    split: DatasetSplit, scaler: ScalerProtocol = StandardScaler()
) -> ScaledDatasetSplit:
    XTrainScaledValues = scaler.fit_transform(split.Train.X)
    XTestScaledValues = scaler.transform(split.Test.X)

    XTrainScaled = pandas.DataFrame(
        XTrainScaledValues, columns=split.Train.X.columns, index=split.Train.X.index
    )

    XTestScaled = pandas.DataFrame(
        XTestScaledValues, columns=split.Test.X.columns, index=split.Test.X.index
    )

    TrainScaledDataset = Dataset(X=XTrainScaled, y=split.Train.y.copy())
    TestScaledDataset = Dataset(X=XTestScaled, y=split.Test.y.copy())

    return ScaledDatasetSplit(
        Train=TrainScaledDataset, Test=TestScaledDataset, Scaler=scaler
    )

# Para almacenar los datos del dataset aplicado PCA.
@dataclass
class PcaDatasetSplit(DatasetSplit):
    Pca: PCA
    Scaler: ScalerProtocol | None = None 

# Aplica PCA al split escalado y retorna el split con PCA aplicado.
def ApplyPCA(
    split: ScaledDatasetSplit,
    explainedVarianceRatioSum: float = 0.95,
    randomState: int = 42
) -> PcaDatasetSplit:

    def GetPCNames(n: int) -> list[str]:
        return [f"PC{i}" for i in range(1, n + 1)]

    pca = PCA(n_components=explainedVarianceRatioSum, random_state=randomState)

    XTrainPCA = pca.fit_transform(split.Train.X)
    XTestPCA = pca.transform(split.Test.X)

    XTrainPcaDf = pandas.DataFrame(
        XTrainPCA, index=split.Train.X.index, columns=GetPCNames(XTrainPCA.shape[1])
    )

    XTestPcaDf = pandas.DataFrame(
        XTestPCA, index=split.Test.X.index, columns=GetPCNames(XTestPCA.shape[1])
    )

    return PcaDatasetSplit(
        Train=Dataset(X=XTrainPcaDf, y=split.Train.y.copy()),
        Test=Dataset(X=XTestPcaDf, y=split.Test.y.copy()),
        Pca=pca,
        Scaler=split.Scaler
    )



SplitLike = ScaledDatasetSplit | PcaDatasetSplit

# Para almacenar los resultados de la regresión logística
@dataclass
class LogisticRegressionResult:
    Model: LogisticRegression
    Predictions: pandas.DataFrame
    Accuracy: float
    Precision: float
    Recall: float
    F1: float
    ConfusionMatrix: np.ndarray
    Iters: list[int] | np.ndarray
    Report: str
    Target: str

# Aplica regresión logística al split escalado y retorna el resultado.
def ApplyLogisticRegression(
    split: SplitLike,
    targetColumn: str,
    randomState: int = 42,
    maxIter: int = 200,
    C: float = 1.0,
    penalty: Literal['l1', 'l2', 'elasticnet'] | None = "l2",
    solver: Literal['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'] = "lbfgs"
) -> LogisticRegressionResult:
    
    XTrain = split.Train.X
    yTrain = split.Train.y[targetColumn]
    XTest = split.Test.X
    yTest = split.Test.y[targetColumn]

    model = LogisticRegression(
        max_iter=maxIter,
        random_state=randomState,
        C=C,
        penalty=penalty,
        solver=solver
    )

    model.fit(XTrain, yTrain)

    yPredTrain = model.predict(XTrain)
    yPredTest = model.predict(XTest)

    yProbaTrain = model.predict_proba(XTrain)
    yProbaTest = model.predict_proba(XTest)

    dfProbaTest = pandas.DataFrame(
        yProbaTest,
        index=XTest.index,
        columns=[f"Class-{cls}-Prob" for cls in model.classes_]
    )

    dfPredTest = pandas.DataFrame(
        {
            "yReal": yTest.values,
            "yPred": yPredTest
        },
        index=XTest.index
    )

    dfPredTest = pandas.concat([dfPredTest, dfProbaTest], axis=1)

    acc = accuracy_score(yTest, yPredTest)
    prec = precision_score(yTest, yPredTest, average="weighted", zero_division=0)
    rec = recall_score(yTest, yPredTest, average="weighted", zero_division=0)
    f1 = f1_score(yTest, yPredTest, average="weighted", zero_division=0)
    cm = confusion_matrix(yTest, yPredTest)
    report = classification_report(yTest, yPredTest)

    return LogisticRegressionResult(
        Model=model,
        Predictions=dfPredTest,
        Accuracy=float(acc),
        ConfusionMatrix=cm,
        Precision=float(prec),
        Recall=float(rec),
        F1=float(f1),
        Iters=model.n_iter_,
        Report=str(report),
        Target=targetColumn
    )

# Para almacenar los resultados de KNN
@dataclass
class KnnResult:
    Model: KNeighborsClassifier
    Predictions: pandas.DataFrame
    Accuracy: float
    Precision: float
    Recall: float
    F1: float
    ConfusionMatrix: np.ndarray
    Report: str
    Target: str
    K: int

def GetBestKForKNN(
    split: SplitLike,
    targetColumn: str,
    kMax: int = 30
) -> int:
    
    XTrain = split.Train.X
    yTrain = split.Train.y[targetColumn]
    XTest = split.Test.X
    yTest = split.Test.y[targetColumn]

    bestK = 1
    bestScore = -1.0

    for k in range(1, kMax + 1):
        model = KNeighborsClassifier(n_neighbors=k)
        model.fit(XTrain, yTrain)

        yPred = model.predict(XTest)
        acc = accuracy_score(yTest, yPred)

        if acc > bestScore:
            bestScore = acc
            bestK = k
    return bestK

def ApplyKNN(
    split: SplitLike,
    targetColumn: str,
    weights: Literal["uniform", "distance"] = "uniform",
    metric: Literal[
        "euclidean", "manhattan", "chebyshev", "minkowski"
    ] = "minkowski"
) -> KnnResult:
    K = GetBestKForKNN(split, targetColumn)
    XTrain = split.Train.X
    yTrain = split.Train.y[targetColumn]
    XTest = split.Test.X
    yTest = split.Test.y[targetColumn]

    model = KNeighborsClassifier(
        n_neighbors=K,
        weights=weights,
        metric=metric
    )

    model.fit(XTrain, yTrain)

    yPred = model.predict(XTest)

    if hasattr(model, "predict_proba"):
        yProba = model.predict_proba(XTest)
        dfProba = pandas.DataFrame(
            yProba,
            index=XTest.index,
            columns=[f"Class-{cls}-Prob" for cls in model.classes_],
        )
    else:
        dfProba = pandas.DataFrame(index=XTest.index)

    dfPred = pandas.DataFrame(
        {
            "yReal": yTest.values,
            "yPred": yPred,
        },
        index=XTest.index
    )

    dfPred = pandas.concat([dfPred, dfProba], axis=1)

    acc = accuracy_score(yTest, yPred)
    prec = precision_score(yTest, yPred, average="weighted", zero_division=0)
    rec = recall_score(yTest, yPred, average="weighted", zero_division=0)
    f1 = f1_score(yTest, yPred, average="weighted", zero_division=0)
    cm = confusion_matrix(yTest, yPred)
    report = classification_report(yTest, yPred)

    return KnnResult(
        Model=model,
        Predictions=dfPred,
        Accuracy=float(acc),
        Precision=float(prec),
        Recall=float(rec),
        F1=float(f1),
        ConfusionMatrix=cm,
        Report=str(report),
        Target=targetColumn,
        K=K
    )

from sklearn.ensemble import RandomForestClassifier

@dataclass
class RandomForestResult:
    Model: RandomForestClassifier
    Predictions: pandas.DataFrame
    Accuracy: float
    Precision: float
    Recall: float
    F1: float
    ConfusionMatrix: np.ndarray
    Report: str
    Target: str

def ApplyRandomForest(
    split: SplitLike,
    targetColumn: str,
    n_estimators: int = 100,
    max_depth: int | None = None,
    criterion: Literal["gini", "entropy", "log_loss"] = "gini",
    randomState: int = 42
) -> RandomForestResult:

    XTrain = split.Train.X
    yTrain = split.Train.y[targetColumn]
    XTest = split.Test.X
    yTest = split.Test.y[targetColumn]

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        criterion=criterion,
        random_state=randomState
    )

    model.fit(XTrain, yTrain)

    yPred = model.predict(XTest)
    yProba = model.predict_proba(XTest)

    dfPred = pandas.DataFrame(
        {"yReal": yTest.values, "yPred": yPred},
        index=XTest.index
    )

    dfProba = pandas.DataFrame(
        yProba,
        columns=[f"Class-{cls}-Prob" for cls in model.classes_],
        index=XTest.index
    )

    dfPred = pandas.concat([dfPred, dfProba], axis=1)

    return RandomForestResult(
        Model=model,
        Predictions=dfPred,
        Accuracy=float(accuracy_score(yTest, yPred)),
        Precision=float(precision_score(yTest, yPred, average="weighted", zero_division=0)),
        Recall=float(recall_score(yTest, yPred, average="weighted", zero_division=0)),
        F1=float(f1_score(yTest, yPred, average="weighted", zero_division=0)),
        ConfusionMatrix=confusion_matrix(yTest, yPred),
        Report=str(classification_report(yTest, yPred)),
        Target=targetColumn
    )

@dataclass
class DecisionTreeResult:
    Model: DecisionTreeClassifier
    Predictions: pandas.DataFrame
    Accuracy: float
    Precision: float
    Recall: float
    F1: float
    ConfusionMatrix: np.ndarray
    Report: str
    Target: str

def ApplyDecisionTree(
    split: SplitLike,
    targetColumn: str,
    max_depth: int | None = None,
    criterion: Literal["gini", "entropy", "log_loss"] = "gini",
    randomState: int = 42
) -> DecisionTreeResult:

    XTrain = split.Train.X
    yTrain = split.Train.y[targetColumn]
    XTest = split.Test.X
    yTest = split.Test.y[targetColumn]

    model = DecisionTreeClassifier(
        max_depth=max_depth,
        criterion=criterion,
        random_state=randomState
    )

    model.fit(XTrain, yTrain)

    yPred = model.predict(XTest)
    yProba = model.predict_proba(XTest)

    dfPred = pandas.DataFrame(
        {"yReal": yTest.values, "yPred": yPred},
        index=XTest.index
    )

    dfProba = pandas.DataFrame(
        yProba,
        columns=[f"Class-{cls}-Prob" for cls in model.classes_],
        index=XTest.index
    )

    dfPred = pandas.concat([dfPred, dfProba], axis=1)

    return DecisionTreeResult(
        Model=model,
        Predictions=dfPred,
        Accuracy=float(accuracy_score(yTest, yPred)),
        Precision=float(precision_score(yTest, yPred, average="weighted", zero_division=0)),
        Recall=float(recall_score(yTest, yPred, average="weighted", zero_division=0)),
        F1=float(f1_score(yTest, yPred, average="weighted", zero_division=0)),
        ConfusionMatrix=confusion_matrix(yTest, yPred),
        Report=str(classification_report(yTest, yPred)),
        Target=targetColumn
    )

@dataclass
class RNAResult:
    Model: Sequential
    Predictions: pandas.DataFrame
    Accuracy: float
    Precision: float
    Recall: float
    F1: float
    ConfusionMatrix: np.ndarray
    Report: str
    Target: str

def ApplyRNA(
    split: SplitLike,
    targetColumn: str,
    epochs: int = 100,
    batchSize: int = 32,
    learningRate: float = 1e-3,
    patience: int = 10
) -> RNAResult:

    XTrain = split.Train.X.to_numpy(dtype=np.float64)
    XTest  = split.Test.X.to_numpy(dtype=np.float64)
    yTrain = split.Train.y[targetColumn].to_numpy(dtype=np.int64)
    yTest  = split.Test.y[targetColumn].to_numpy(dtype=np.int64)

    model = Sequential([
        Dense(10, activation="relu", input_dim=XTrain.shape[1]),
        Dropout(0.2),
        Dense(1, activation="sigmoid")
    ])

    model.compile(
        optimizer=Adam(learning_rate=learningRate),
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )

    earlyStopping = EarlyStopping(
        monitor="val_loss",
        patience=patience,
        restore_best_weights=True
    )

    model.fit(
        XTrain, yTrain,
        epochs=epochs,
        batch_size=batchSize,
        validation_data=(XTest, yTest),
        callbacks=[earlyStopping],
        verbose=0
    )

    yProba = model.predict(XTest).ravel()
    yPred = (yProba > 0.5).astype(int)

    dfPred = pandas.DataFrame(
        {
            "yReal": yTest,
            "yPred": yPred,
            "Class-1-Prob": yProba
        },
        index=split.Test.X.index
    )

    return RNAResult(
        Model=model,
        Predictions=dfPred,
        Accuracy=float(accuracy_score(yTest, yPred)),
        Precision=float(precision_score(yTest, yPred, zero_division=0)),
        Recall=float(recall_score(yTest, yPred, zero_division=0)),
        F1=float(f1_score(yTest, yPred, zero_division=0)),
        ConfusionMatrix=confusion_matrix(yTest, yPred),
        Report=str(classification_report(yTest, yPred)),
        Target=targetColumn
    )

from sklearn.metrics import roc_auc_score

def GenerateMetricsTable(
    splitList: list[SplitLike],
    targetColumn: str,
    includeRNA: bool = True
) -> pandas.DataFrame:

    results = []

    for idx, split in enumerate(splitList, start=1):

        splitType = "PCA" if isinstance(split, PcaDatasetSplit) else "SCALED"
        scalerName = DetectScaler(split.Scaler).value

        # Modelos
        lr  = ApplyLogisticRegression(split, targetColumn)
        knn = ApplyKNN(split, targetColumn)
        dt  = ApplyDecisionTree(split, targetColumn)
        rf  = ApplyRandomForest(split, targetColumn)

        row = {
            "Index": idx,
            "Type": splitType,
            "Scaler": scalerName,

            # Logistic Regression
            "LR_Accuracy": lr.Accuracy,
            "LR_F1": lr.F1,

            # KNN
            "KNN_K": knn.K,
            "KNN_Accuracy": knn.Accuracy,
            "KNN_F1": knn.F1,

            # Decision Tree
            "DT_Accuracy": dt.Accuracy,
            "DT_F1": dt.F1,

            # Random Forest
            "RF_Accuracy": rf.Accuracy,
            "RF_F1": rf.F1,
        }

        if includeRNA:
            rna = ApplyRNA(split, targetColumn)
            row.update({
                "RNA_Accuracy": rna.Accuracy,
                "RNA_F1": rna.F1
            })

        results.append(row)

    return pandas.DataFrame(results)

# Función para graficar la matriz de confusión
def PlotConfusionMatrix(
    cm,
    classNames: Sequence[str] | None = None,
    title: str = "Confusion Matrix"
):
    # Si classNames es None → usar "auto"
    xticks = classNames if classNames is not None else "auto"
    yticks = classNames if classNames is not None else "auto"

    plt.figure(figsize=(6, 5))
    sns.heatmap(
        cm,
        annot=True,
        fmt="d",
        cmap="Blues",
        xticklabels=xticks,
        yticklabels=yticks
    )
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(title)
    plt.tight_layout()
    plt.show()

# Utilidades para detección de tipos de split
@dataclass(frozen=True)
class SplitTypeInfo:
    IsPCA: bool
    IsScaled: bool
    IsRaw: bool

# Detecta el tipo de split (PCA, Escalado, Crudo)
def DetectSplitType(split) -> SplitTypeInfo:
    isPca = isinstance(split, PcaDatasetSplit)
    isScaled = isinstance(split, ScaledDatasetSplit)
    isRaw = not isPca and not isScaled

    return SplitTypeInfo(
        IsPCA=isPca,
        IsScaled=isScaled,
        IsRaw=isRaw
    )

def PlotAccuracyBySplit(
    dfMetrics: pandas.DataFrame,
    title: str = "Accuracy por Modelo y Split"
):
    # Modelos y columnas asociadas
    modelCols = {
        "LR": "LR_Accuracy",
        "KNN": "KNN_Accuracy",
        "DT": "DT_Accuracy",
        "RF": "RF_Accuracy",
        "RNA": "RNA_Accuracy",
    }

    # Número de splits
    nSplits = dfMetrics.shape[0]
    splitLabels = [f"Split {i+1}" for i in range(nSplits)]

    models = list(modelCols.keys())
    nModels = len(models)

    x = np.arange(nModels)
    width = 0.8 / nSplits  # ancho dinámico

    plt.figure(figsize=(12, 6))

    for i in range(nSplits):
        accuracies = [
            dfMetrics.loc[i, col]
            for col in modelCols.values()
        ]

        plt.bar(
            x + i * width,
            accuracies,
            width,
            label=splitLabels[i]
        )

    plt.xticks(x + width * (nSplits - 1) / 2, models)
    plt.ylabel("Accuracy")
    plt.ylim(0, 1.05)
    plt.title(title)
    plt.legend(title="Splits")
    plt.grid(axis="y", linestyle="--", alpha=0.6)

    plt.tight_layout()
    plt.show()

def _PrintModelMetrics(
    result,
    showConfusionMatrix: bool = True
):
    print(f"Accuracy : {result.Accuracy:.4f}")
    print(f"Precision: {result.Precision:.4f}")
    print(f"Recall   : {result.Recall:.4f}")
    print(f"F1 Score : {result.F1:.4f}")

    if showConfusionMatrix:
        print("Confusion Matrix:")
        PlotConfusionMatrix(
            result.ConfusionMatrix,
            classNames=None,
            title="Confusion Matrix"
        )

    print("Classification Report:")
    print(result.Report)

def EvaluateDatasetSplits(
    splitList: list[SplitLike],
    targetColumn: str,
    includeRNA: bool = True,
    showConfusionMatrix: bool = True
):
    for idx, split in enumerate(splitList, start=1):

        splitType = "PCA" if isinstance(split, PcaDatasetSplit) else "SCALED"
        scalerName = DetectScaler(split.Scaler).value

        header = (
            f"SPLIT {idx} | Type={splitType} | "
            f"Scaler={scalerName} | "
            f"Features={split.Train.X.shape[1]}"
        )
        ShowTitleBox(f"EVALUACIÓN DE MODELOS - {header}", color=ConsoleColor.MAGENTA, boxLineStyle= TitleBoxLineStyle.BLOCK)

        print("\n🔹 LOGISTIC REGRESSION")
        lr = ApplyLogisticRegression(split, targetColumn)
        _PrintModelMetrics(lr, showConfusionMatrix)

        print("\n🔹 KNN")
        knn = ApplyKNN(split, targetColumn)
        print(f"Best K: {knn.K}")
        _PrintModelMetrics(knn, showConfusionMatrix)

        print("\n🔹 DECISION TREE")
        dt = ApplyDecisionTree(split, targetColumn)
        _PrintModelMetrics(dt, showConfusionMatrix)

        print("\n🔹 RANDOM FOREST")
        rf = ApplyRandomForest(split, targetColumn)
        _PrintModelMetrics(rf, showConfusionMatrix)

        if includeRNA:
            print("\n🔹 RNA (Red Neuronal Artificial)")
            rna = ApplyRNA(split, targetColumn)
            _PrintModelMetrics(rna, showConfusionMatrix)


# TITANIC DATASET 1️⃣ Business Understanding - Entender el negocio

| Columna         | Descripción                                                                                                                                                  |
| --------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------ |
| **PassengerId** | Identificador único asignado a cada pasajero en el dataset. No tiene significado real en el viaje, solo sirve como índice.                                   |
| **Survived**    | Indica si el pasajero **sobrevivió (1)** o **murió (0)**. Es la variable objetivo en la mayoría de proyectos de Machine Learning sobre este dataset.         |
| **Pclass**      | Clase del boleto del pasajero: **1 = Primera clase**, **2 = Segunda clase**, **3 = Tercera clase**. Representa una medida indirecta de nivel socioeconómico. |
| **Name**        | Nombre completo del pasajero, generalmente incluye tratamiento o título social (Mr., Mrs., Miss, etc.).                                                      |
| **Sex**         | Sexo del pasajero: **male** (hombre) o **female** (mujer).                                                                                                   |
| **Age**         | Edad del pasajero en años. Puede contener valores faltantes. Los niños pequeños pueden aparecer con valores decimales.                                       |
| **SibSp**       | Número de **hermanos (siblings)** y **cónyuges (spouses)** que viajaban con el pasajero en el Titanic.                                                       |
| **Parch**       | Número de **padres (parents)** e **hijos (children)** que viajaban con el pasajero.                                                                          |
| **Ticket**      | Código del boleto del pasajero. Puede incluir letras y números; no cumple un formato estándar.                                                               |
| **Fare**        | Tarifa pagada por el pasajero por su boleto. El valor depende de la clase y otras condiciones del viaje.                                                     |
| **Cabin**       | Número de cabina asignada. Muchos valores están vacíos porque la mayoría de pasajeros no tenía cabina asignada (especialmente en 3ra clase).                 |
| **Embarked**    | Puerto de embarque: **C = Cherbourg**, **Q = Queenstown**, **S = Southampton**.                                                                              |


# TITANIC DATASET 2️⃣ Data preparation - Preparación de los datos
- Se descargan los archivos CSV que son Train, Test y los datos de Test de supervivencia. 
- Se eliminan columnas innecesarias. "Cabin", "PassengerId", "Ticket", "Name"
- Se modifica las letras de embarque por los nombres de cada puerto real. "S" por "Southampton", "C" por "Cherbourg" y "Q" por "Queenstown".
- Se aplica One Hot Encoding a la columna Embarked dado que es nominal y no tiene un orden especifico.
- Se aplica One Hot Encoding a la columna Sex ya que es nominal y no depende de ningún orden específico.
- Se creó una nueva columna para obtener el tamaño de la familia "FamilySize", es el total que incluye conyuges, hijos, padres y la persona.
- Se imputó la columna Age por el promedio de los datos de Train.
- Se imputó la columna Fare por el promedio de los datos de Train. Ya que en los datos de Test existía valores null.
- Se elimina columnas no deseadas a propósito del usuario.
- Finalmente se crean el DatasetSplit de datos que contiene los Datasets con la información "X" y "y" tanto para Train como para Test.
- Se muestra la tabla de correlación completa. Las tablas de correlación fuerte y débil con un umbral de 0.4. Si las correlaciones son mayores o iguales al umbral se muestran en la tabla de correlación fuerte caso contrario no se muestran. Si las correlaciones son menores al umbral se muestran en la tabla de correlación débil caso contrario no se muestran. 
 
Con un umbral de 0.4 existe correlación fuerte moderada del 26% en las columnas y el 74% no tiene correlación fuerte. 

- Finalmente quedan listos los datos listos para utilizar en un entrenamiento con algoritmo supervisado y posteriormente hacer los tests.


In [ ]:
DOWNLOADS_DIR = "Temp"
TITANIC_TEST_DATA_URL = "https://github.com/UIDE-Tareas/6-Herramientas-Data-Science-Tarea1/raw/refs/heads/main/Datasets/TitanicTest.csv"
TITANIC_TEST_GENDER_SUB_DATA_URL = "https://github.com/UIDE-Tareas/6-Herramientas-Data-Science-Tarea1/raw/refs/heads/main/Datasets/TitanicGenderSub.csv"
TITANIC_TRAIN_DATA_URL = "https://github.com/UIDE-Tareas/6-Herramientas-Data-Science-Tarea1/raw/refs/heads/main/Datasets/TitanicTrain.csv"
TITANIC_TEST_FILENAME = os.path.join(DOWNLOADS_DIR, "TitanicTest.csv")
TITANIC_TRAIN_FILENAME = os.path.join(DOWNLOADS_DIR, "TitanicTrain.csv")
TITANIC_TEST_GENDER_SUB_FILENAME = os.path.join(DOWNLOADS_DIR, "TitanicGenderSub.csv")


# Carga y preprocesa el dataset de Titanic, retornando el split Train/Test. 
def LoadTitanicDatasetSplit(toDrop: list[str]=[])-> DatasetSplit:
    toDrop = [c for c in toDrop if c != "Survived"]
    DownloadFile(TITANIC_TEST_DATA_URL, TITANIC_TEST_FILENAME, printInfo=False)
    DownloadFile(TITANIC_TRAIN_DATA_URL, TITANIC_TRAIN_FILENAME, printInfo=False)
    DownloadFile(TITANIC_TEST_GENDER_SUB_DATA_URL, TITANIC_TEST_GENDER_SUB_FILENAME, printInfo=False)
    dfTrain = pd.read_csv(TITANIC_TRAIN_FILENAME)
    dfTest = pd.read_csv(TITANIC_TEST_FILENAME)
    dfTestGenderSub = pd.read_csv(TITANIC_TEST_GENDER_SUB_FILENAME)

    toDiscard = ["Cabin", "PassengerId", "Ticket", "Name"]
    dfTrain = dfTrain.drop(columns=toDiscard)
    dfTest  = dfTest.drop(columns=toDiscard)

    dtypeMap = {
        "Survived": "int64",
        "Pclass": "int64",
        "Sex": "string",
        "Age": "float64",
        "SibSp": "int64",
        "Parch": "int64",
        "Fare": "float64",
        "Embarked": "string"
    }

    dfTrain = dfTrain.astype(dtypeMap)
    dfTest  = dfTest.astype({col: dtypeMap[col] for col in dfTest.columns})

    mapEmbarked = {
        "S": "Southampton",
        "C": "Cherbourg",
        "Q": "Queenstown"
    }
    dfTrain["Embarked"] = dfTrain["Embarked"].replace(mapEmbarked)
    dfTest["Embarked"]  = dfTest["Embarked"].replace(mapEmbarked)

    dfTrain = pd.get_dummies(dfTrain, columns=["Embarked"], prefix="Embarked")
    dfTest  = pd.get_dummies(dfTest, columns=["Embarked"], prefix="Embarked")

    dfTrain = pd.get_dummies(dfTrain, columns=["Sex"], prefix="Sex", drop_first=False)
    dfTest  = pd.get_dummies(dfTest, columns=["Sex"], prefix="Sex", drop_first=False)

    dfTrain["FamilySize"] = dfTrain["SibSp"] + dfTrain["Parch"] + 1
    dfTest["FamilySize"] = dfTest["SibSp"] + dfTest["Parch"] + 1

    meanAge = dfTrain["Age"].mean()
    dfTrain["Age"] = dfTrain["Age"].fillna(meanAge)
    dfTest["Age"]  = dfTest["Age"].fillna(meanAge)

    meanFare = dfTrain["Fare"].mean()
    dfTrain["Fare"] = dfTrain["Fare"].fillna(meanAge)
    dfTest["Fare"]  = dfTest["Fare"].fillna(meanAge)

    NormalizeColumnNames(dfTrain)
    NormalizeColumnNames(dfTest)

    dfTrain = DropColumns(dfTrain, toDrop, inplace=True)
    dfTest  = DropColumns(dfTest, toDrop, inplace=True)

    XTrain = dfTrain.drop(columns=["Survived"])
    yTrain = dfTrain[["Survived"]].copy()
    XTest  = dfTest
    yTest  = dfTestGenderSub[["Survived"]].copy()
    return DatasetSplit(
        Train=Dataset(X=XTrain, y=yTrain),
        Test=Dataset(X=XTest, y=yTest)
    )

ShowTitleBox("EXPLORACIÓN INICIAL TITANIC DATASET", color=ConsoleColor.MAGENTA, boxLineStyle= TitleBoxLineStyle.BLOCK)
ShowTitleBox("CARGANDO EL DATASET", color=ConsoleColor.CYAN, boxLineStyle= TitleBoxLineStyle.SIMPLE)
split0 = LoadTitanicDatasetSplit()
ShowTitleBox("INFO DEL DATASET", color=ConsoleColor.CYAN, boxLineStyle= TitleBoxLineStyle.SIMPLE)
title = "Titanic Dataset Split"
ShowDatasetSplitInfo(split0, title, headQty=10)

ShowTitleBox("MATRIZ DE CORRELACIÓN", color=ConsoleColor.CYAN, boxLineStyle= TitleBoxLineStyle.SIMPLE)
title = "TITANIC Data original TRAIN"
CORR_UMBRAL = 0.4
_=ShowDfCorrelation(split0.Train.X, title, level=CorrelationType.ALL, showTable=True, figsize=(10,8), annotate=True, umbral=CORR_UMBRAL)
_=ShowDfCorrelation(split0.Train.X, title, level=CorrelationType.STRONG, showTable=False, figsize=(10,8), annotate=True, umbral=CORR_UMBRAL)
_=ShowDfCorrelation(split0.Train.X, title, level=CorrelationType.WEAK, showTable=False, figsize=(10,8), annotate=True, umbral=CORR_UMBRAL)



# TITANIC DATASET 3️⃣ y 4️⃣ Modeling - Evaluating - Aplicación del modelo y evaluación

In [ ]:
splitEscaled = ScaleDatasetSplit(  LoadTitanicDatasetSplit())
splitWithPCA = ApplyPCA( ScaleDatasetSplit(  LoadTitanicDatasetSplit()))
splitEscaledMinMax = ScaleDatasetSplit(  LoadTitanicDatasetSplit(), scaler=CreateScaler(ScalerType.MIN_MAX))
splitWithPCAMinMax = ApplyPCA( ScaleDatasetSplit(  LoadTitanicDatasetSplit(), scaler=CreateScaler(ScalerType.MIN_MAX)))
allSplits = [splitEscaled, splitWithPCA, splitEscaledMinMax, splitWithPCAMinMax]

EvaluateDatasetSplits(
    splitList=allSplits,
    targetColumn="Survived",
    includeRNA=True,
    showConfusionMatrix=True
)

ShowTitleBox("GENERACIÓN DE TABLA DE MÉTRICAS", color=ConsoleColor.CYAN, boxLineStyle= TitleBoxLineStyle.SIMPLE)
dfMetrics = GenerateMetricsTable(allSplits, targetColumn="Survived", includeRNA=True)
display(dfMetrics)

ShowTitleBox("GRÁFICA DE ACCURACY POR SPLIT Y MODELO", color=ConsoleColor.CYAN, boxLineStyle= TitleBoxLineStyle.SIMPLE)
PlotAccuracyBySplit(dfMetrics)

# 🏁 Resultados 🏁

Se tiene 4 datasets para probar:

- Dataset original escalado con StandardScaler.
- Dataset original escalado con Standard Scaler y aplicado PCA.
- Dataset original escalado con MinMaxScaler.
- Dataset original escalado con MinMax Scaler y aplicado PCA.


Aplicamos cada uno de los modelos y generamos la tabla de métricas. Con la tabla de métricas mostramos un gráfico de barras con las accuracy de cada modelo por cada DatasetSplit.

Las mejores opciones para predecir son KNN, LR y RNA para este dataset. RF y DT dan resultados pobres. Utilizando MinMax Scaler se tiene el mejor rendimiento tanto con Scalado como con PCA.

El modelo ganador para cualquier dataset escalado o con PCA es la *RNA*. Que da un resultado en accuracy de 0.99 con escalado y 0.95 con PCA. 


